In [1]:
# Kütüphaneler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale     # standardizasoyon işlemleri
from sklearn.preprocessing import StandardScaler    # standardizasoyon işlemleri
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [2]:
df = pd.read_csv("C:\python_py\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])

y = df["Salary"]

x__ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")
x = pd.concat([x__ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

- Bu bölümde diğerlerinden farklı olarak bir standartlaştırma işlemi uygulayacağız. Bunu yapmamızın amacı bazı algoritmalar heterojen bazı algoritmalar ise homojen veri setleri üzerinde daha iyi çalışır. Yapay sinir ağları da homojen veri setleri üzerinde daha iyi çalışır. Çünkü hataların optimizasyonu noktasında yapılan türevleme işlmelerinde ortaya çıkmış olan aykırı değerlerin ortaya çıkarttığı bazı problemler söz konusudur. Bundan dolayı yapay sinir ağlarını standartlaştırma işlemleri sonrasında kullanmak daha sağlıklıdır. 

# Standartlaştırma İşlemi


In [5]:
scaler_x_train = StandardScaler().fit(x_train).transform(x_train)

scaler_x_test = StandardScaler().fit(x_test).transform(x_test)

# MODEL

In [7]:
mlp_model = MLPRegressor().fit(scaler_x_train , y_train)    # ml = multi layer -> çok katmanlı

# TAHMİN

In [11]:
y_pred = mlp_model.predict(scaler_x_test)
y_pred[0:5]

array([19.88550774, 36.06307195, 69.50857331,  8.3240672 , 28.8407236 ])

In [13]:
# Test setinin ilk hata oranına bakalım

RMSE = np.sqrt(mean_squared_error(y_pred ,y_test))
RMSE

658.9635182593942

# MODEL TUNİNG (DOĞRULAMA)

In [15]:
mlp_params = {"alpha":[0.1 , 0.01 , 0.02 , 0.001 , 0.0001] , "hidden_layer_sizes":[(10,20) , (5,5) , (100,100)]}  # lambdalar yani ceza parametreleri
                                                                                                                  # ve gizli katman sayıları (10,20) -> iki tane sayı olduğu için 2 tane gizli katman var.
                                                                                                                  #                                     bir katmanda 10 nöron diğerinde 20 nöron var

# her bir alfa değerini tek tek tüm hidden_layer_size ile deneyecek

In [29]:
mlp_cv_model = GridSearchCV(mlp_model , mlp_params , cv=10 , verbose=2).fit(scaler_x_train , y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 20); total time=   0.0s
[CV] END ...............alpha=0.1, hidden_layer_sizes=(5, 5); total time=   0.0s
[CV] END ...............alpha=0.1, hidden_laye

In [30]:
mlp_cv_model.best_params_       # optimum parametreler

{'alpha': 0.001, 'hidden_layer_sizes': (100, 100)}

In [31]:
# FİNAL MODELİ

mlp_tuned = MLPRegressor(alpha=0.001 , hidden_layer_sizes=(100,100)).fit(scaler_x_train , y_train)

In [32]:
y_pred = mlp_tuned.predict(scaler_x_test)

RMSE = np.sqrt(mean_squared_error(y_pred , y_test))
RMSE

# İlk tahminimiz 658.9635182593942'idi. Hata oranı büyük ölçüde düştü

352.60711812952167